In [41]:
import matplotlib.pyplot as plt
from data_generation import MEANS, VARIANCES, KEY, generate_mv_data, NUM_SAMPLES
from plot_utils import plot_scatter, plot_circles, plot_metrics
from train_utils import SimpleDnn, Encoder, Linear_Encoder, create_loaders
import torch
from torch.utils.data import TensorDataset
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
import torch.nn.functional as F
import pandas as pd
from sklearn.datasets import make_circles
import matplotlib.image as image
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
import plotly.express as px
from csv_utils import format_csv
from pathlib import Path
import weightwatcher as ww

BATCH_SIZE = 32

# Linear Data Example

1. Generate linear data from three Gaussians

In [2]:
# generate data
data, labels = generate_mv_data(KEY, MEANS, VARIANCES, NUM_SAMPLES, 3)

# format labels and set up dataloaders
labels_one_hot = F.one_hot(torch.Tensor(np.hstack((np.array(labels)))).to(torch.int64), num_classes=3).float() 
data_linear = TensorDataset(
                torch.Tensor(np.vstack((np.array(data)))), labels_one_hot
            )
train_loader_linear, val_loader_linear = create_loaders(data = data_linear, ratio = 0.8, num_workers = 0, shuffle_train = True, shuffle_val = False, batch_size = 32)

2023-04-03 15:27:27.888 | INFO     | train_utils:create_loaders:44 - Creating dataloaders with 80/20train/test split 🔪
2023-04-03 15:27:27.889 | INFO     | train_utils:create_loaders:70 - Successfully created train and validation loader 🤗


### 1. Train 'unbroken' model (simple feed-forward network)

In [10]:
# define model and logger
simple_dnn = SimpleDnn(Encoder(input_dim = 2, output_dim = 3), task_type = 'classification')
csv_logger = CSVLogger(save_dir='metrics_csv', name = 'linear')
# train model on linear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50, log_every_n_steps=20)
trainer.fit(model=simple_dnn, train_dataloaders = train_loader_linear, val_dataloaders = val_loader_linear)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory metrics_csv/linear/version_0/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type          | Params
------------------------------------------
0 | encoder | Encoder       | 63    
1 | f1      | BinaryF1Score | 0     
------------------------------------------
63        Trainable params
0         Non-trainable params
63        Total params
0.000     Total estimated model params size (MB)
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/lightning_fabric/loggers/csv_logs.py:188: UserWarning: Experiment log

Epoch 49: 100%|██████████| 38/38 [00:00<00:00, 177.47it/s, v_num=0]         

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 38/38 [00:00<00:00, 173.16it/s, v_num=0]


In [11]:
# format, since csv logger captures metrics on step basis
format_csv('metrics_csv/linear/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/linear_data.csv'))

### 2. Broken model: No activation functions

In [43]:
# DNN without ReLUs
simple_dnn_linear = SimpleDnn(Linear_Encoder(input_dim = 2, output_dim = 3), task_type = 'classification')
csv_logger = CSVLogger(save_dir='metrics_csv', name = 'linear_no_relu')
# train model on linear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50, log_every_n_steps=20)
trainer.fit(model=simple_dnn_linear, train_dataloaders = train_loader_linear, val_dataloaders = val_loader_linear)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


NameError: name 'train_loader_linear' is not defined

In [13]:
# format, since csv logger captures metrics on step basis
format_csv('metrics_csv/linear_no_relu/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/linear_data_no_relu.csv'))

# Nonlinear Data Example

In [44]:
data_circles, label_circles = make_circles(n_samples=NUM_SAMPLES, factor=0.5, noise=0.05)
# concentric circles
data_tensor_circles = TensorDataset(
                torch.Tensor(data_circles), F.one_hot(torch.Tensor(label_circles).to(torch.int64), num_classes=2).float() 
            )
train_loader_circles, val_loader_circles = create_loaders(data = data_tensor_circles, ratio = 0.8,  batch_size = 32, num_workers = 0,
                                                        shuffle_train = True, shuffle_val = False)

2023-04-08 15:56:39.352 | INFO     | train_utils:create_loaders:58 - Creating dataloaders with 80/20train/test split 🔪
2023-04-08 15:56:39.370 | INFO     | train_utils:create_loaders:84 - Successfully created train and validation loader 🤗


### 1. Train 'unbroken' model (simple feed-forward network)

In [16]:
simple_dnn = SimpleDnn(Encoder(input_dim = 2, output_dim = 2), task_type = 'classification')
csv_logger = CSVLogger(save_dir='metrics_csv', name = 'circles')
# train model on nonlinear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50)
trainer.fit(model=simple_dnn, train_dataloaders = train_loader_circles, val_dataloaders = val_loader_circles)
format_csv('metrics_csv/circles/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/circle_data.csv'))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: metrics_csv/circles

  | Name    | Type          | Params
------------------------------------------
0 | encoder | Encoder       | 52    
1 | f1      | BinaryF1Score | 0     
------------------------------------------
52        Trainable params
0         Non-trainable params
52        Total params
0.000     Total estimated model params size (MB)


/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model

Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 171.54it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 161.16it/s, v_num=0]


### 2. Broken model: No activation functions

In [18]:
# DNN without ReLUs
simple_dnn_linear = SimpleDnn(Linear_Encoder(input_dim = 2, output_dim = 2), task_type = 'classification')
# train model on nonlinear data
csv_logger = CSVLogger(save_dir='metrics_csv', name = 'circles_no_relu')
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50)
trainer.fit(model=simple_dnn_linear, train_dataloaders = train_loader_circles, val_dataloaders = val_loader_circles)
format_csv('metrics_csv/circles_no_relu/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/circle_data_no_relu.csv'))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: metrics_csv/circles_no_relu

  | Name    | Type           | Params
-------------------------------------------
0 | encoder | Linear_Encoder | 52    
1 | f1      | BinaryF1Score  | 0     
-------------------------------------------
52        Trainable params
0         Non-trainable params
52        Total params
0.000     Total estimated model params size (MB)


/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model

Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 142.19it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 132.34it/s, v_num=0]


### 3. Broken model: Frozen weights

In [148]:
simple_dnn_frozen_weights = SimpleDnn(Encoder(input_dim = 2, output_dim = 2), task_type = 'classification')

num_params = len([param for param in simple_dnn.parameters()])
for i, param in zip(range(num_params), simple_dnn.parameters()):
    if i in [num_params-1, num_params -2] :
        param.requires_grad = False

csv_logger = CSVLogger(save_dir='metrics_csv', name = 'circles_frozen')
# train model on nonlinear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50)
trainer.fit(model=simple_dnn_frozen_weights, train_dataloaders = train_loader_circles, val_dataloaders = val_loader_circles)
format_csv('metrics_csv/circles_frozen/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/circle_data_frozen.csv'))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type          | Params
------------------------------------------
0 | encoder | Encoder       | 52    
1 | f1      | BinaryF1Score | 0     
------------------------------------------
52        Trainable params
0         Non-trainable params
52        Total params
0.000     Total estimated model params size (MB)


Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 112.61it/s, v_num=0]         

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 106.09it/s, v_num=0]


### 4. Broken model: Frozen Bias

In [7]:
simple_dnn = SimpleDnn(Encoder(input_dim = 2, output_dim = 2), task_type = 'classification')

num_params = len([param for param in simple_dnn.parameters()])
for i, param in zip(range(num_params), simple_dnn.parameters()):
    if i == num_params-1 :
        param.requires_grad = False

csv_logger = CSVLogger(save_dir='metrics_csv', name = 'circles_frozen_bias')
# train model on nonlinear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50)
trainer.fit(model=simple_dnn, train_dataloaders = train_loader_circles, val_dataloaders = val_loader_circles)
format_csv('metrics_csv/circles_frozen_bias/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/circle_data_frozen_bias.csv'))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory metrics_csv/circles_frozen_bias/version_0/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type          | Params
------------------------------------------
0 | encoder | Encoder       | 52    
1 | f1      | BinaryF1Score | 0     
------------------------------------------
50        Trainable params
2         Non-trainable params
52        Total params
0.000     Total estimated model params size (MB)
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/lightning_fabric/loggers/csv_logs.py:188: UserWarning: E

/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model

Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 56.09it/s, v_num=0] 

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 53.67it/s, v_num=0]


### 5. Broken model: Trainloader returns single example

In [45]:
simple_dnn = SimpleDnn(Encoder(input_dim = 2, output_dim = 2), task_type = 'classification')
train_loader_circles_broken, val_loader_circles = create_loaders(data = data_tensor_circles, ratio = 0.8,  batch_size = 32, num_workers = 0,
                                                        shuffle_train = True, shuffle_val = False, subset_broken_train = True)

csv_logger = CSVLogger(save_dir='metrics_csv', name = 'circles_dataloader_broken')
# train model on nonlinear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50)
trainer.fit(model=simple_dnn, train_dataloaders = train_loader_circles_broken, val_dataloaders = val_loader_circles)
format_csv('metrics_csv/circles_dataloader_broken/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/circle_dataloader_broken.csv'))

2023-04-08 15:56:42.822 | INFO     | train_utils:create_loaders:58 - Creating dataloaders with 80/20train/test split 🔪
2023-04-08 15:56:42.827 | INFO     | train_utils:create_loaders:84 - Successfully created train and validation loader 🤗
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type          | Params
------------------------------------------
0 | encoder | Encoder       | 52    
1 | f1      | BinaryF1Score | 0     
------------------------------------------
52        Trainable params
0         Non-trainable params
52        Total params
0.000     Total estimated model params size (MB)


Epoch 49: 100%|██████████| 7/7 [00:00<00:00, 100.66it/s, v_num=0]          

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 7/7 [00:00<00:00, 93.07it/s, v_num=0] 


In [79]:
import cleanlab 
from skorch import NeuralNetClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from cleanlab.filter import find_label_issues
import numpy as np

In [60]:
model_skorch = NeuralNetClassifier(simple_dnn.encoder)
cl = cleanlab.classification.CleanLearning(model_skorch)
#label_issues = cl.find_label_issues(data_circles, label_circles )

In [80]:
def sigmoid(x):
    """Average first and last element of a 1-D array"""
    return 1/(1 + np.exp(-x))

In [81]:
np.apply_along_axis(sigmoid, 0, pred_probs)

array([[0.81302947, 0.8002704 ],
       [0.67569894, 0.6858361 ],
       [0.91713905, 0.8993359 ],
       [0.6729651 , 0.68356246],
       [0.9066113 , 0.8917656 ],
       [0.88529915, 0.87522846],
       [0.77103543, 0.7775052 ],
       [0.9265095 , 0.91764635],
       [0.8879582 , 0.8861431 ],
       [0.7696639 , 0.7584154 ],
       [0.7683457 , 0.7757159 ],
       [0.65968424, 0.6614381 ],
       [0.8863094 , 0.86825585],
       [0.8891023 , 0.87101865],
       [0.9138049 , 0.89840204],
       [0.7503066 , 0.7424206 ],
       [0.71412134, 0.7275472 ],
       [0.6695623 , 0.6732817 ],
       [0.7320839 , 0.7278798 ],
       [0.77855647, 0.78822684],
       [0.748246  , 0.74039614],
       [0.8846367 , 0.87468857],
       [0.9429745 , 0.92597425],
       [0.7512115 , 0.74058384],
       [0.7627599 , 0.7617192 ],
       [0.7785881 , 0.7655811 ],
       [0.9135059 , 0.8973663 ],
       [0.6608126 , 0.66873616],
       [0.6499082 , 0.6554777 ],
       [0.7778929 , 0.76610947],
       [0.

In [82]:
pred_probs_logits = cross_val_predict(
    model_skorch,
    np.float32(data_circles),
    label_circles,
    cv=3,
    method="predict_proba",
)

pred_probs = np.apply_along_axis(sigmoid, 0, pred_probs_logits)

predicted_labels = pred_probs.argmax(axis=1)
acc = accuracy_score(label_circles, predicted_labels)
print(f"Cross-validated estimate of accuracy on held-out data: {acc}")

ranked_label_issues = find_label_issues(
    label_circles,
    pred_probs,
    return_indices_ranked_by="self_confidence",
)

print(f"Cleanlab found {len(ranked_label_issues)} label issues.")
print(f"Top 15 most likely label errors: \n {ranked_label_issues[:15]}")

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.0538       0.5373       -0.2381  0.0263
      2       -0.0122       0.5075       -0.2815  0.0099
      3       -0.0678       0.4627       -0.3207  0.0101
      4       -0.1164       0.4627       -0.3566  0.0072
      5       -0.1600       0.4328       -0.3901  0.0104
      6       -0.1999       0.4030       -0.4217  0.0120
      7       -0.2371       0.3582       -0.4515  0.0109
      8       -0.2718       0.3582       -0.4798  0.0089
      9       -0.3045       0.3284       -0.5070  0.0100
     10       -0.3357       0.3284       -0.5330  0.0091
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.0473       0.3582        0.0241  0.0069
      2       -0.0169       0.3582       -0.0338  0.0066
      3       -0.0714       0.3433       -0.0842  0.0087
      4       -0.1193       0.3

In [100]:
# concentric circles with label mixup
data_circles, label_circles = make_circles(n_samples=NUM_SAMPLES, factor=0.5, noise=0.05)
corrupted_indices = np.random.choice(range(len(label_circles)), size = 20, replace = False)
label_circles[corrupted_indices] = abs(label_circles[corrupted_indices] - 1)

data_tensor_circles = TensorDataset(
                torch.Tensor(data_circles), F.one_hot(torch.Tensor(label_circles).to(torch.int64), num_classes=2).float() 
            )
# save for plotting
df = pd.DataFrame({'x1': data_circles[:,0], 'x2': data_circles[:,1], 'labels': label_circles})
df.to_csv('label_mixup/circles_corrupted.csv', index = False)

train_loader_circles, val_loader_circles = create_loaders(data = data_tensor_circles, ratio = 0.8,  batch_size = 32, num_workers = 0,
                                                        shuffle_train = True, shuffle_val = False)

2023-04-08 16:29:29.811 | INFO     | train_utils:create_loaders:58 - Creating dataloaders with 80/20train/test split 🔪
2023-04-08 16:29:29.841 | INFO     | train_utils:create_loaders:84 - Successfully created train and validation loader 🤗


In [104]:
simple_dnn = SimpleDnn(Encoder(input_dim = 2, output_dim = 2), task_type = 'classification')
csv_logger = CSVLogger(save_dir='metrics_csv', name = 'circles_label_mixup')
# train model on nonlinear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 100)
trainer.fit(model=simple_dnn, train_dataloaders = train_loader_circles, val_dataloaders = val_loader_circles)
format_csv('metrics_csv/circles_label_mixup/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/circles_label_mixup_data.csv'))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type          | Params
------------------------------------------
0 | encoder | Encoder       | 52    
1 | f1      | BinaryF1Score | 0     
------------------------------------------
52        Trainable params
0         Non-trainable params
52        Total params
0.000     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 13/13 [00:00<00:00, 113.79it/s, v_num=0]         

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 13/13 [00:00<00:00, 92.24it/s, v_num=0] 


In [98]:
px.scatter(         df,
                    x='x1',
                    y='x2',
                    color='labels',
                    size_max=60,
                    color_continuous_scale = px.colors.sequential.Peach,
                    opacity = 0.5,
                    )

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'x1=%{x}<br>x2=%{y}<br>labels=%{marker.color}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': array([1, 0, 0, ..., 1, 0, 0]),
                         'coloraxis': 'coloraxis',
                         'opacity': 0.5,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([-0.52881484, -0.17325255, -0.30698274, ...,  0.44295477,  0.83076864,
                          -0.76529524]),
              'xaxis': 'x',
              'y': array([ 0.30670762, -0.96102615,  0.98063511, ..., -0.05329088,  0.4078448 ,
                          -0.69652268]),
              'yaxis': 'y'}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'labels'}},
                             'colorscale': [[0.0, 'rgb(253, 224, 197)'],
                                            [0.16666666666666666, 'rgb(250, 203,
                                            166)'], [0.3333333333333333, 'rgb(248,
                                            181, 139)'], [0.5, 'rgb(245, 158,
                                            114)'], [0.6666666666666666, 'rgb(242,
                                            133, 93)'], [0.8333333333333334,
                                            'rgb(239, 106, 76)'], [1.0, 'rgb(235,
                                            74, 64)']]},
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x1'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'x2'}}}
})

In [107]:
model_skorch = NeuralNetClassifier(simple_dnn.encoder)
cl = cleanlab.classification.CleanLearning(model_skorch)

pred_probs_logits = cross_val_predict(
    model_skorch,
    np.float32(data_circles),
    label_circles,
    cv=3,
    method="predict_proba",
)

pred_probs = np.apply_along_axis(sigmoid, 0, pred_probs_logits)

predicted_labels = pred_probs.argmax(axis=1)
acc = accuracy_score(label_circles, predicted_labels)
print(f"Cross-validated estimate of accuracy on held-out data: {acc}")

ranked_label_issues = find_label_issues(
    label_circles,
    pred_probs,
    return_indices_ranked_by="self_confidence",
)

print(f"Cleanlab found {len(ranked_label_issues)} label issues.")
print(f"Top 15 most likely label errors: \n {ranked_label_issues[:15]}")

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.9552           nan  0.0101
      2           nan       0.9552           nan  0.0081
      3           nan       0.9552           nan  0.0059
      4           nan       0.9552           nan  0.0091
      5           nan       0.9552           nan  0.0104
      6           nan       0.9552           nan  0.0080
      7           nan       0.9552           nan  0.0102
      8           nan       0.9552           nan  0.0079
      9           nan       0.9552           nan  0.0070
     10           nan       0.9552           nan  0.0075
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.9403           nan  0.0077
      2           nan       0.9403           nan  0.0118
      3           nan       0.9403           nan  0.0096
      4           nan       0.9

In [118]:
len(corrupted_indices)

20

In [116]:
len(list(set(corrupted_indices).intersection(ranked_label_issues)))

9

In [110]:
label_circles_allegedly_corrupted = label_circles.copy()
label_circles_allegedly_corrupted[ranked_label_issues] = 2

In [111]:
df2 = pd.DataFrame({'x1': data_circles[:,0], 'x2': data_circles[:,1], 'labels': label_circles_allegedly_corrupted})

In [112]:
px.scatter(         df2,
                    x='x1',
                    y='x2',
                    color='labels',
                    size_max=60,
                    color_continuous_scale = px.colors.sequential.Peach,
                    opacity = 0.5,
                    )

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'x1=%{x}<br>x2=%{y}<br>labels=%{marker.color}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': array([0, 0, 1, ..., 0, 1, 1]),
                         'coloraxis': 'coloraxis',
                         'opacity': 0.5,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([-0.71079146,  0.78699149, -0.55901743, ..., -0.78858728, -0.39174613,
                           0.28942423]),
              'xaxis': 'x',
              'y': array([-0.72462306, -0.50461388, -0.02240178, ...,  0.59545306, -0.42433183,
                           0.4736757 ]),
              'yaxis': 'y'}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'labels'}},
                             'colorscale': [[0.0, 'rgb(253, 224, 197)'],
                                            [0.16666666666666666, 'rgb(250, 203,
                                            166)'], [0.3333333333333333, 'rgb(248,
                                            181, 139)'], [0.5, 'rgb(245, 158,
                                            114)'], [0.6666666666666666, 'rgb(242,
                                            133, 93)'], [0.8333333333333334,
                                            'rgb(239, 106, 76)'], [1.0, 'rgb(235,
                                            74, 64)']]},
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x1'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'x2'}}}
})

## 6. Broken models with weightwatcher

In [298]:
class CustomEncoder(nn.Module):
    """
    Simple encoder
    """

    def __init__(self, input_dim: int, output_dim: int) -> None:
        """Setup model layers"""
        super().__init__()
        self.dense1 =  nn.Linear(input_dim, 10)
        self.dense2 = nn.Linear(10, 9)
        self.dense3 = nn.Linear(10, 8)
        self.dense4 = nn.Linear(10, 7)
        self.dense5 = nn.Linear(10, 6)
        self.dense6 = nn.Linear(10, output_dim)
        self.relu = nn.ReLU()

        
        self.output_dim = output_dim
        self.input_dim = input_dim

    def forward(self, x: torch.Tensor) -> nn.Sequential:
        """Forward pass through layers"""
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.relu(x)
        x = self.dense4(x)
        x = self.relu(x)
        x = self.dense5(x)
        x = self.relu(x)
        x = self.dense6(x)
        return x

simple_dnn = SimpleDnn(CustomEncoder(input_dim = 2, output_dim = 2), task_type = 'classification')
train_loader_circles_broken, val_loader_circles = create_loaders(data = data_tensor_circles, ratio = 0.8,  batch_size = 32, num_workers = 0,
                                                        shuffle_train = True, shuffle_val = False, subset_broken_train = True)

csv_logger = CSVLogger(save_dir='metrics_csv', name = 'circles_dataloader_broken')
# train model on nonlinear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50)
trainer.fit(model=simple_dnn, train_dataloaders = train_loader_circles_broken, val_dataloaders = val_loader_circles)
format_csv('metrics_csv/circles_dataloader_broken/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/circle_dataloader_broken.csv'))

2023-04-06 15:05:59.754 | INFO     | train_utils:create_loaders:44 - Creating dataloaders with 80/20train/test split 🔪
2023-04-06 15:05:59.773 | INFO     | train_utils:create_loaders:70 - Successfully created train and validation loader 🤗
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type          | Params
------------------------------------------
0 | encoder | CustomEncoder | 492   
1 | f1      | BinaryF1Score | 0     
------------------------------------------
492       Trainable params
0         Non-trainable params
492       Total params
0.002     Total estimated model params size (MB)


Epoch 49: 100%|██████████| 7/7 [00:00<00:00, 70.24it/s, v_num=0]            

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 7/7 [00:00<00:00, 59.35it/s, v_num=0]


In [310]:
simple_dnn_everything_ok = SimpleDnn(CustomEncoder(input_dim = 2, output_dim = 2), task_type = 'classification')
train_loader_circles, val_loader_circles = create_loaders(data = data_tensor_circles, ratio = 0.8,  batch_size = 32, num_workers = 0,
                                                        shuffle_train = True, shuffle_val = False, subset_broken_train = False)

csv_logger = CSVLogger(save_dir='metrics_csv', name = 'circles_dataloader_broken')
# train model on nonlinear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50)
trainer.fit(model=simple_dnn, train_dataloaders = train_loader_circles, val_dataloaders = val_loader_circles)
format_csv('metrics_csv/circles_dataloader_broken/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/circle_dataloader_broken.csv'))

2023-04-06 15:08:12.919 | INFO     | train_utils:create_loaders:44 - Creating dataloaders with 80/20train/test split 🔪
2023-04-06 15:08:12.926 | INFO     | train_utils:create_loaders:70 - Successfully created train and validation loader 🤗
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type          | Params
------------------------------------------
0 | encoder | CustomEncoder | 492   
1 | f1      | BinaryF1Score | 0     
------------------------------------------
492       Trainable params
0         Non-trainable params
492       Total params
0.002     Total estimated model params size (MB)


Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 101.04it/s, v_num=0]         

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 95.04it/s, v_num=0] 
